<a href="https://colab.research.google.com/github/vggls/M.Sc._AI_Thesis/blob/main/experiments/xrays_resnet34/HAAS_Results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Imports**

In [ ]:
pip install grad-cam

In [ ]:
import torch
from torchvision  import datasets, transforms
from pytorch_grad_cam import GradCAM, HiResCAM

In [ ]:
# custom written code
from xrays import create_datasets
from haas import Haas, HA_image
#from densenet121 import _DenseNet121_

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **2. Download from Kaggle**

In [ ]:
! mkdir ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download tawsifurrahman/covid19-radiography-database

100% 777M/778M [00:34<00:00, 24.3MB/s]
100% 778M/778M [00:34<00:00, 23.7MB/s]


In [ ]:
! unzip covid19-radiography-database.zip

# **3. Test dataset**

In [ ]:
# to be applied to training data
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),          #recommended size
    transforms.RandomHorizontalFlip(),      #data augmentation
    transforms.RandomVerticalFlip(),        #data augmentation
    transforms.RandomRotation(degrees=20),  #data augmentation
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor([0.5, 0.5, 0.5]), torch.Tensor([0.5, 0.5, 0.5]))
])
# to be applied to validation and test data
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor([0.5, 0.5, 0.5]), torch.Tensor([0.5, 0.5, 0.5]))
])

In [ ]:
train_dataset, validation_dataset, test_dataset = create_datasets(train_transforms, test_transforms)

# **4. ResNet34**

In [ ]:
drive_path = './drive/MyDrive/Colab_Notebooks/dataset_models/Covid-19_Radiography_Dataset/Models/'

In [ ]:
resnet34 = torch.load(drive_path + 'xrays_resnet34.pt')
resnet34 = resnet34.cuda()
resnet34 = resnet34.eval()

In [ ]:
resnet34_gradcam_instance = GradCAM(model=resnet34, target_layers=[resnet34.layer4[2].conv2], use_cuda=True)
resnet34_hirescam_instance = HiResCAM(model=resnet34, target_layers=[resnet34.layer4[2].conv2], use_cuda=True)

In [ ]:
print('-->GradCAM')
gradcam_score, _, _, _ = Haas(test_dataset, resnet34, resnet34_gradcam_instance)
print('HAAS score:', gradcam_score)
print('---------------------------')
print('-->HiResCAM')
hirescam_score, _, _, _ = Haas(test_dataset, resnet34, resnet34_hirescam_instance)
print('HAAS score:', hirescam_score)

-->GradCAM
No. of test images:  2119
Total time: 72.83775806427002 secs
Accuracy over original images:  0.95
Accuracy over HA images:  0.82
HAAS score: 0.86
---------------------------
-->HiResCAM
No. of test images:  2119
Total time: 72.65495181083679 secs
Accuracy over original images:  0.95
Accuracy over HA images:  0.64
HAAS score: 0.67
